In [ ]:
import os
import cv2
import numpy as np
import random
from keras.models import load_model

# in some model MSE has been used in other a combination of MSE and Focal loss has been used
# testing has been done in 2 different ways, method 1: we pass the entire img for the model to predict,
# method 2: we resize img to 1024x1024. train on random patches of 512x512. test on 4 non overlapping 512x512 patches for all test imgs
# after basic MCNN we start splitting data into train and val.
# we also printed errors in each test examples sorted based on which gave more error and analysed

In [ ]:
import os
import cv2
import numpy as np
def load_dataset(image_dir, density_dir, input_size=(768, 1024), output_size=(192, 256)):
    images = []
    densities = []
    image_files = os.listdir(image_dir)
    for image_file in image_files:
        image_path = os.path.join(image_dir, image_file)
        density_path = os.path.join(density_dir, image_file.replace('.jpg', '.npy'))
        
        # load & preprocess img
        image = cv2.imread(image_path)
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = cv2.resize(image, input_size)
        image = image / 255.0
        
        # load & preprocess density map
        density = np.load(density_path)
        old_h, old_w = density.shape
        density = cv2.resize(density, output_size)

        # rescale density to preserve total count
        scaling_factor = (old_h * old_w) / (output_size[0] * output_size[1])
        density = density * scaling_factor

        density = density[..., np.newaxis]

        images.append(image)
        densities.append(density)

    return np.array(images), np.array(densities)

image_dir= 'crowd_wala_dataset/train_data/images'
density_dir = 'density_maps/train_data_sigma_9_new2/'
X, Y = load_dataset(image_dir, density_dir)
print(X.shape, Y.shape)

(400, 1024, 768, 3) (400, 256, 192, 1)


In [ ]:
# train_test_split
# from sklearn.model_selection import train_test_split

# X_train, X_val, Y_train, Y_val = train_test_split(X, Y, test_size=0.1, random_state=42)

# print(f"train shape: {X_train.shape}, {Y_train.shape}")
# print(f"val shape: {X_val.shape}, {Y_val.shape}")

In [3]:
for i in Y_train:
    print(np.sum(i))

218.1647
216.49251
204.52306
81.704414
25.231754
150.39014
29.986843
128.2212
128.78969
116.53313
207.87088
83.893265
136.19829
29.672665
144.23419
86.61216
104.33358
82.69711
69.71112
166.8442
266.00052
57.620567
299.8041
96.57166
26.173079
40.040688
42.728138
148.54416
236.33632
26.362684
104.09512
51.181454
40.46443
140.16698
113.19511
47.391148
212.86404
113.3421
243.89394
147.17593
41.833145
172.65565
227.73302
101.90081
55.517296
49.21595
161.56868
149.0795
145.30496
29.589256
24.104342
176.76689
90.1107
289.51712
281.09967
110.25662
232.48566
97.231544
157.94691
37.013268
26.089153
411.05252
142.2281
85.6056
259.64682
11.064617
134.7923
40.637623
147.20763
47.5861
297.18546
164.20546
120.32141
26.558329
98.55382
195.72226
235.77989
104.39292
46.48849
149.68816
14.504221
135.89484
41.44818
172.46696
362.70834
264.1342
91.945694
57.486546
41.843437
176.85207
63.933533
105.587845
100.5764
115.39583
79.01911
128.81725
196.22646
122.88568
338.15067
89.44609
41.944016
137.28067
548.88

In [1]:
# define your model here

In [ ]:
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam

model.compile(optimizer=Adam(learning_rate=1e-5), loss='mse')  

early_stopping = EarlyStopping(
    monitor='val_loss',  
    patience=5,
    verbose=1,
    mode='min'      
)

reduce_lr = ReduceLROnPlateau(
    monitor='val_loss',
    factor=0.5,       
    patience=3,
    verbose=1,
    min_lr=1e-7
)


history = model.fit(
    X, Y,
    batch_size=8,
    epochs=10,
    callbacks=[early_stopping, reduce_lr],
    shuffle=True       
)

In [ ]:
# model.save("crowd_counting_model2.keras")
# from tensorflow.keras.models import load_model
model = load_model("new_model6.keras")

In [ ]:
import os
import scipy.io
import cv2
import numpy as np
import matplotlib.pyplot as plt

def load_ground_truth(mat_file):
    data = scipy.io.loadmat(mat_file)
    points = data['image_info'][0,0][0,0][0]
    return points

def preprocess_image(image_path):
    image = cv2.imread(image_path)
    image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
    image = cv2.resize(image, (768, 1024))  
    image = image / 255.0
    image = np.expand_dims(image, axis=0)
    return image

test_image_dir = 'crowd_wala_dataset/test_data/images/'
test_gt_dir = 'crowd_wala_dataset/test_data/ground_truth/'
test_files = os.listdir(test_image_dir)

In [ ]:
# Start evaluation
total_absolute_error = 0
squared_error = 0
total_samples = 0

for file in test_files:
    image_path = os.path.join(test_image_dir, file)
    gt_path = os.path.join(test_gt_dir, "GT_" + file.replace(".jpg", ".mat"))
    
    # load, preprocess, predict
    test_image = preprocess_image(image_path)
    predicted_density = model.predict(test_image)
    predicted_density = np.squeeze(predicted_density)
    predicted_count = predicted_density.sum()
    
    # gt count
    points = load_ground_truth(gt_path)
    gt_count = len(points)
    
    # Compare
    error = abs(predicted_count - gt_count)
    total_absolute_error += error
    squared_error += (error*error)
    total_samples += 1
    
    print(f"Image: {file}")
    print(f"Predicted Count: {predicted_count:.2f}")
    print(f"Ground Truth Count: {gt_count}")
    print(f"Absolute Error: {error:.2f}\n")

mae = total_absolute_error / total_samples
mse = squared_error / total_samples
print(f"Mean Absolute Error on test set: {mae:.2f}")